In [1]:
import pandas as pd
import numpy as np

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier 

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split 
from sklearn import metrics

### Classificando usando TF-IDF

In [2]:
dataset  = pd.read_csv('treated_dataset.csv')

In [3]:
dataset.sample(5)

,tweet_text,sentiment
69335,chor outr vez :),1
157132,ir escol :),1
467572,:( vo,0
479561,5s 5 tip problem difer :( ruim indic ning,0
211550,sim :) fal transform vid tamb,1


In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(lowercase = False)
data_vectorized = tfidf.fit_transform(dataset.tweet_text.apply(lambda tweet: np.str_(tweet)))

In [5]:
SEED = 20

treino, teste, classe_treino, classe_teste = train_test_split(
    data_vectorized,
    dataset.sentiment,
    test_size = 0.2,
    random_state = SEED)

In [6]:
modelo = LogisticRegression(solver = 'lbfgs', multi_class='auto',max_iter = 500)
modelo.fit(treino,classe_treino)
result = modelo.predict(teste)
print("Accuracy:",metrics.accuracy_score(classe_teste, result))

Accuracy: 0.75865


In [7]:
modelo = DecisionTreeClassifier(max_depth = 5)
modelo.fit(treino,classe_treino)
result = modelo.predict(teste)
print("Accuracy:",metrics.accuracy_score(classe_teste, result))

Accuracy: 0.59304
